<a href="https://colab.research.google.com/github/utsavdatta-git/keras_chatbots/blob/master/Chatbot_using_seq2seq_model_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
def setup_google_drive():
  # Code to download file into Colaboratory:
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive
  
def download_personality_data(drive):
  #Downloading 3 Personality chat dataset files, change the ids with your own files and locations
  id = '1tIQYqkdyN58w3MSyqydCU07qkJiZNo3i'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('qna_chitchat_the_comic.tsv') 

  id = '1aqTcA0N6UQl3jIeMMrxdF-aAlvtL9NVg'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('qna_chitchat_the_professional.tsv')

  id = '1e5EJ5DJnFlhfAPOmU5jg5AVfzaH3BURb'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('qna_chitchat_the_friend.tsv')

## 1.2. Preprocess data (Personality chat datasets)

Data preprocessing techniques used:

1. **De-capitalization** - changing all word case to lowercase. 

    **Justification** - The goal of de-capitalization is to make all words for word embedding model and all words of seq2seq model input change to lowercase to make them uniform across the board. This reduces the size of vocab and increases performance during model creation and chat execution without losing word contexts.
    
2. **Word Contraction Removal** - Removing word contractions as much as possible

    **Justification** - Removing word contractions helps to keep the vocab of word embedding model to an optimum size. For e.g. there could be many forms of a single phrase in the dataset like "are not" may exist as  {"aren't", "are not"}. Our goal is to keep only one of these in the vocab. Also, it helps to eradicate the special character " ' '" without losing the word meaning and standardize common words.
    
3. **Punctuation Removal**  -  Remove all punctuations from the input text

     **Justification** - Punctutations like "," and "." do not hold any extra information for our chatbot which mainly words on a series of whole words. Hence we have removed them. Also, having punctuations in the word embedding model may lead to unwanted errors due to presence of an unexpected punctutations which might be present in the validation text but not in the text on which the word embedding model was trained. As we know, fasttext (word embedding we have used here) will store n-grams of whole words but not of special characters.
     
4. **Remove Numbers and Non-alphanumeric characters** 

    **Justification** -  The justification for this is also to remove special characters which do not necessarily add to the meaning of a word in sentences and keep the word representations effective. Although numbers may have some role to define context (e.g. I want that 4 my birthday), in this assignmrnt we have assumed that the user will enter whole words (for instead of 4)
     
4. **Lemmatization** - Keeping only a meaningful lemma of various forms of a word

     **Justification** - The goal of lemmatization is to keep only a meaningful word lemma in the fastext vocab and help cover a wide range of word forms by a single representation. This helps reduce missing words in the vocab during chat bot execution. For e.g. we intend to represent {"running", "runs"} by the word "run".
     
5. **Stemming** - Reducing inflection in words to their root forms

    **Justification** - The main goal of Stemming is to keep only the root form of a word in the fastext model to further reduce the size of fastext vocab and help it cover a large no. of word forms by a single stem represenation. For e.g. "troubl" will represent {"trouble, "troubling", "troubled"...}. 
    
    
After preprocessing, we get a list containing tokenized words with pre-processing steps applied e.g a text = "like a sentence e.g "I am running, he's running too" will be transformed as  ['i', 'am', 'run', 'he', 'is', 'run', 'too']

In [0]:
#-----------------------------------------------------------------------------
# Function name: text_preprocess
# Purpose: This fuction pre-processes an input text through a series of pre-processing pipelines.
#          It has been implemented as a function to be used as a common function 
#          to pre-process texts at any stage of the assignment             
# Input: a string of text
# Output: preprocessed list of words
#-----------------------------------------------------------------------------
def text_preprocess(text):
  #Strip leading and trailing spaces and De-capitalize
  text = str(text).strip().lower()
  #These are just common English contractions as much as possible
  contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
  #Removing contractions
  text_cleaned = " ".join([contraction_dict.get(x,x) for x in text.split()])
  
  #Removing punctuations, removing numbers and mupliple spaces
  CUSTOM_FILTERS = [lambda x: x, strip_punctuation, strip_numeric, strip_multiple_whitespaces,strip_non_alphanum]
  dialogues_list_preprocessed = preprocess_string(text_cleaned, CUSTOM_FILTERS)
 
  stemmer = PorterStemmer()
  
  lemmatizer = WordNetLemmatizer()
  
  #Stemming and Lemmatizing
  dialogues_list_preprocessed = [lemmatizer.lemmatize(x) for x in dialogues_list_preprocessed]
  dialogues_list_preprocessed = [stemmer.stem(x) for x in dialogues_list_preprocessed]
  dialogues_list_preprocessed = [x.replace("’","") for x in dialogues_list_preprocessed]
  
  return dialogues_list_preprocessed

# 2 - Model Implementation

## 2.1. Word Embeddings

We have used **FastText** word embedding model.

**Justification** - We would like to avoid runtime errors during chatbot execution which might happen due to non-existence of certain unexpected words in the word2vec model. Unlike CBOW and SkipGram, FastText stores original word representation in its model along with n-grams of the word too. Due to this FastText model has an additional ability to return word vectors for out-of-vocabulary words. This is the reason we chose FastText. 

### 2.1.1. Download Dataset for Word Embeddings



We have used Cornell Movie Lines Dataset for building Word Embeddings. This corpus contains a large no. of conversations (220,579 conversational exchanges between 10,292 pairs of movie characters). Therefore, this corpus will have more life-like conversations and as it contains 9,035 characters from 617 movies, it will have diffrent contexts of words from different situations. Due to this, the model will have large and effective dataset to learn from and create meaningful word represenataions.

In [0]:
# Download Dataset for Word Embeddings
def download_and_read_word_embed_data(drive):
  id = '1Gn7fjrCZ_Orqgh-BpmIkS-r_JnehbHji'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('movie_lines.txt')

  # Read the Cornell Movie Dialog Corpus (movie_lines.txt) which has all the movie dialogues
  diag = pd.read_csv("movie_lines.txt",delimiter = "\+\+\+\$\+\+\+", names=[0,1,2,3,4],engine='python')
  diag.head(2)
  dialogues_list = list(diag[4])
  return dialogues_list

### 2.1.2. Data Preprocessing for Word Embeddings

### 2.1.3. Build Word Embeddings Model

**Hyperparameters:**

1. size: We have used a popular size of word vectors. We think this is good for having effective word representation.

2. window: We have a used a popular window size to learn from. We think this is good for undestanding context of a word effectively.

In [0]:
#-----------------------------------------------------------------------------
# Function name: word2vec_fasttext
# Purpose: This function takes a list of sentences and builds a word2vec (fasttext) representation of words
# Input: a list of lists containing sentences e.g. [["sentence 1","sentence 2","sentence 3"]]
# Output: fasttext word2vec model trained over the input
#-----------------------------------------------------------------------------
def word2vec_fasttext(sentences):
  ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)
  return ft_sg_model

### 2.1.4. Train Word Embeddings Model

In [0]:
# Call the word2vec_fasttext function to create the model using the sentences appearing in Cornell- 
# movie dialogue dataset
def train_word2vec_model(dialogues_list):
  final_model_input=[]
  for x in dialogues_list:
    final_model_input.append(text_preprocess(x))
  ft_sg_model = word2vec_fasttext(final_model_input)
  return ft_sg_model

### 2.1.5. Save Word Embeddings Model

In [0]:
# Save trained fasttext model to google drive
def save_word2vec_model(model):
  model.save("/content/gdrive/My Drive/chatbot/word2vec_fasttext_model")

### 2.1.6. Load Word Embeddings Model

In [0]:
# Load fasttext model for use from google drive
def load_word2vec_model():
  word2vec_fasttext_model_saved = FastText.load("/content/gdrive/My Drive/chatbot/word2vec_fasttext_model")
  return word2vec_fasttext_model_saved

## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*We have done experiments to choose the correct value for the hyperparameter "learning rate"". Given below are some of the trend in loss over 6000 epochs for diffrent learning rates (we used a lot of different learning rates but we have shown here the best ones that lead to highest loss). This experiment was done on the "qna_chitchat_the_friend"*

![alt text](...)

We chose learning rate = 0.001

Additionally, we did some experiments to choose the correct and optimum value for no. of hidden layers. The plots for different values of no. of hidden layers w.r.t epoch and Loss achieved is given below. Since seq2seq model training is an infrequent exercise, we chose 256 hidden layers (for friend and comic datasets) and 128 layers for Professional datasets as they achieved comparatively lower loss over 6000 epochs.

![alt text](...%0A9c6I6PqHQRkAAAAASUVORK5CYII=)


### 2.2.2. Build Seq2Seq Model

In [0]:
# get word2vec vectors for words in question
def get_vectors_q(sentence, num_dic, max_input_words_amount,ft_sg_model_saved):
    # tokenise the sentence
    tokenized_sentence = word_tokenize(sentence) 
    diff = max_input_words_amount - len(tokenized_sentence)
    
    # add paddings if the word is shorter than the maximum number of words    
    for x in range(diff):
        tokenized_sentence.append('_P_')
        
    data = tokens_to_ids(tokenized_sentence,num_dic,"q",ft_sg_model_saved)           
    return data

# get token index vector of answer
def get_vectors_a(sentence, num_dic):    
    tokenized_sentence = [sentence]
    data = tokens_to_ids(tokenized_sentence,num_dic,"a",ft_sg_model_saved)  
    return data
    

# convert tokens to index
def tokens_to_ids(tokenized_sentence,num_dic,input_type,ft_sg_model_saved):
    ids = []
    if input_type == "q":
      for token in tokenized_sentence:
            ids.append(ft_sg_model_saved[token])
    if input_type == "a":      
      for token in tokenized_sentence:
          if token in num_dic:
              ids.append(num_dic[token])
          else:
              ids.append(num_dic["_U_"])
    return ids


# generate a batch data for training/testing
def make_batch(seq_data, num_dic, max_input_words_amount,ft_sg_model_saved):
    input_batch = []
    output_batch = []
    target_batch = []
    for seq in seq_data:
        # Input for encoder cell, convert question to vector
        input_data = get_vectors_q(seq[0], num_dic, max_input_words_amount,ft_sg_model_saved)           
        # Input for decoder cell, Add '_B_' at the beginning of the sequence data
        output_data = [num_dic['_B_']] 
        output_data += get_vectors_a(seq[1],num_dic)
        # Output of decoder cell (Actual result), Add '_E_' at the end of the sequence data
        target = get_vectors_a(seq[1],num_dic)
        target.append(num_dic['_E_'])                             
        # Convert each token vector to one-hot encode data
        input_batch.append(input_data)
        output_batch.append(np.eye(len(num_dic))[output_data])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

#-----------------------------------------------------------------------------
# Function name: enumerate_question_and_answer
# Purpose: This function creates a dictionary of words to be used to fetch word2vec represenation of training data and 
#          decides input/output vector length of the seq2seq model
# Input: dataset filename, e.g. qna_chitchat_the_comic.tsv
# Output: Sequence data, Generate unique tokens list from qas.json
#-----------------------------------------------------------------------------
def enumerate_question_and_answer(dataset_filename):
  seq_data = []
  whole_words = []
  max_input_words_amount = 0
  max_output_words_amount = 1


  df = pd.read_csv(dataset_filename, sep="\t")
  for index, row in df.iterrows():
      question = ' '.join(text_preprocess(row[0]))
      answer = row[1]
      seq_data.append( [question,answer] )
      # we need to tokenise question    
      tokenized_q = word_tokenize(question)

      # we do not need to tokenise answer (because we implement N to One model)
      # make a list with only one element (whole sentence)
      tokenized_a = [answer]
      # add question list and answer list (one element)
      whole_words += tokenized_q
      whole_words += tokenized_a
      # we need to decide the maximum size of input word tokens
      max_input_words_amount = max(len(tokenized_q), max_input_words_amount)


  # we now have a vacabulary list
  unique_words = list(set(whole_words))

  # adding special tokens in the vocabulary list    
  # _B_: Beginning of Sequence
  # _E_: Ending of Sequence
  # _P_: Padding of Sequence - for different size input
  # _U_: Unknown element of Sequence - for different size input

  unique_words.append('_B_')
  unique_words.append('_E_')
  unique_words.append('_P_')
  unique_words.append('_U_')
  
  unique_words = sorted(unique_words)

  num_dic = {n: i for i, n in enumerate(unique_words)}
  dic_len = len(num_dic)
  
  return seq_data, num_dic, dic_len, max_input_words_amount, max_output_words_amount, unique_words

### 2.2.3. Train Seq2Seq Model

In [0]:
#-----------------------------------------------------------------------------
# Function name: build_train_save_seq2seq_model
# Purpose: This function build the Seq2Seq model and trains it for a set no. of epochs and then saves it in google drive
# Input: dataset filename, e.g. qna_chitchat_the_comic.tsv, fasttext model (to fetch word vectors for words in questions)
# Output: None
#-----------------------------------------------------------------------------
def build_train_save_seq2seq_model(filename,ft_sg_model_saved, no_of_layers):
  seq_data, num_dic, dic_len, max_input_words_amount, max_output_words_amount,unique_words = enumerate_question_and_answer(filename)
  ### Neural Network Model
  tf.reset_default_graph()
  ### Setting Hyperparameters
  learning_rate = 0.001
  n_hidden = no_of_layers

  n_class = dic_len
  n_input = dic_len

  # encoder/decoder shape = [batch size, time steps, input size]
  enc_input = tf.placeholder(tf.float32, [None, None, 100],name="enc_input_train")
  dec_input = tf.placeholder(tf.float32, [None, None, n_input],name="dec_input_train")
  # target shape = [batch size, time steps]
  targets = tf.placeholder(tf.int64, [None, None],name="targets_train")


  # Encoder Cell
  with tf.variable_scope('encode'):
      enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
      enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

      outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                              dtype=tf.float32)
  # Decoder Cell
  with tf.variable_scope('decode'):
      dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
      dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

      # [IMPORTANT] Setting enc_states as inital_state of decoder cell
      outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                              initial_state=enc_states,
                                              dtype=tf.float32)

  model = tf.layers.dense(outputs, n_class, activation=None, name="model")

  cost = tf.reduce_mean(
              tf.nn.sparse_softmax_cross_entropy_with_logits(
                  logits=model, labels=targets, name="softmax"),name="cost")

  optimizer = tf.train.AdamOptimizer(learning_rate,name="adam").minimize(cost,name="optimizer")


  ### Training Model
  init=tf.global_variables_initializer()
  sess = tf.Session()
  sess.run(init)

  # Generate a batch data
  input_batch, output_batch, target_batch = make_batch(seq_data, num_dic, max_input_words_amount, ft_sg_model_saved)

  total_epoch = 6000

  for epoch in range(total_epoch):
      _,loss = sess.run([optimizer, cost],
                         feed_dict={enc_input: input_batch,
                                    dec_input: output_batch,
                                    targets: target_batch})
      if epoch % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  save_seq2seq_model(sess,filename[:filename.index(".")])

### 2.2.4. Save Seq2Seq Model

In [0]:
# Function to save TensorFlow model in Google Drive
def save_seq2seq_model(sess,filename):
  saver = tf.train.Saver()
  save_path = saver.save(sess, "/content/gdrive/My Drive/chatbot/"+filename+"/seq2seq_model_"+filename+".ckpt")
  print("Model saved in path: %s" % save_path)

### 2.2.5. Load Seq2Seq Model

In [0]:
# Function to restore TensorFlow model from Google Drive
def load_seq2seq_model(i, graph,sess):
    new_saver = tf.train.import_meta_graph('/content/gdrive/My Drive/chatbot/'+lisdataset[i]+'/seq2seq_model_'+lisdataset[i]+'.ckpt.meta')
    new_saver.restore(sess, '/content/gdrive/My Drive/chatbot/'+lisdataset[i]+'/seq2seq_model_'+lisdataset[i]+'.ckpt')
    enc_input_train = graph.get_tensor_by_name("enc_input_train:0")
    dec_input_train = graph.get_tensor_by_name("dec_input_train:0")
    targets_train = graph.get_tensor_by_name("targets_train:0")
    model_train = graph.get_tensor_by_name("model/BiasAdd:0")
    prediction = tf.argmax(model_train,2)
    return enc_input_train, dec_input_train, targets_train, prediction

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
# Function to start the chat session. It takes in the question, predicts an answer and return the answer
def start_chatting(sess, sentence, i,enc_input_train, dec_input_train, targets_train, prediction):
  seq_data_orig = [sentence, '_U_']
  seq_data, num_dic, dic_len, max_input_words_amount, max_output_words_amount, unique_words = enumerate_question_and_answer(lisdataset[i]+".tsv")
  input_batch, output_batch, target_batch = make_batch([seq_data_orig], num_dic, max_input_words_amount,ft_sg_model_saved)
  result = sess.run(prediction,
                         feed_dict={enc_input_train: input_batch,
                                    dec_input_train: output_batch,
                                    targets_train: target_batch}) 
  decoded = [unique_words[i] for i in result[0]]
  #Remove anything after '_E_'        
  if "_E_" in decoded:
      end = decoded.index('_E_')
      translated = ' '.join(decoded[:end])
  else :
      translated = ' '.join(decoded[:])
  return translated

## 3.2. Change Personality

To change personality to "Comic" enter "0"

To change personality to "Friend" enter "1"

To change personality to "Professional" enter "2" 

## 3.3. Save chat log

In [0]:
# Function to save a chat log text file and write that chat log in Google Drive
def save_chat_log(file_handle,string):  
  file_handle.write(string)
def write_chat_log():
  with open('./chat_log.txt','r') as f_read:
    text = f_read.readlines()
  with open('/content/gdrive/My Drive/chatbot/chat_log.txt', 'w') as f:
    f.write("".join(text))

## 3.5. Execute program

### 3.5.1. Execute program - training mode

In [18]:
# import all the libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import json
import re
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
import pandas as pd
import nltk
from gensim.models import FastText
nltk.download('wordnet')
nltk.download('punkt')
from gensim.parsing.preprocessing import preprocess_string,strip_punctuation, strip_numeric, strip_multiple_whitespaces, strip_non_alphanum
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from google.colab import drive
!pip install paramiko
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


drive.mount('/content/gdrive',force_remount=True)

driver = setup_google_drive()

download_personality_data(driver)

dialogue_list = download_and_read_word_embed_data(driver)

#These 2 statements are commented as word2vec model is already trained and uploaded 
ft_sg_model = train_word2vec_model(dialogue_list)

save_word2vec_model(ft_sg_model)

ft_sg_model_saved = load_word2vec_model()

#This statement is commented as seq2seq model is already trained and uploaded
saving_models = [build_train_save_seq2seq_model(file, ft_sg_model_saved, 256) for \
                 file in ["qna_chitchat_the_comic.tsv","qna_chitchat_the_friend.tsv"]]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
saving_models = [build_train_save_seq2seq_model(file, ft_sg_model_saved, 128) for \
                 file in ["qna_chitchat_the_professional.tsv"]]

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [27]:
tf.logging.set_verbosity(tf.logging.ERROR)
with open("./chat_log.txt", 'w') as f:
  tf.reset_default_graph()
  graph = tf.get_default_graph()
  print("<BEGIN CHAT>")
  print("<To Change Personality at any time, enter 0 for Comic, 1 for Friend, 2 for Professional, ""Exit"" to Exit>")
  save_chat_log(f,"<BEGIN CHAT>"+"\n")
  lisdataset=["qna_chitchat_the_comic","qna_chitchat_the_friend","qna_chitchat_the_professional"]
  i=2
  ex=""
  change_personality=False
  while(not change_personality):
    if (ex.lower()=="exit"):
      break
    change_personality=True
    tf.reset_default_graph()
    graph = tf.get_default_graph()
    while (change_personality):
      with tf.Session() as sess:
        init=tf.global_variables_initializer()
        sess.run(init)      
        enc_input_train, dec_input_train, targets_train, prediction= load_seq2seq_model(i, graph,sess)
        inputtext = input("User: ")  
        save_chat_log(f,"User: "+inputtext+"\n")
        if (inputtext=="0"):
          i=0
          print("<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper())
          save_chat_log(f,"<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper()+"\n")
        if (inputtext=="1"):
          i=1
          print("<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper())
          save_chat_log(f,"<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper()+"\n")
        if (inputtext=="2"):
          i=2
          print("<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper())
          save_chat_log(f,"<PERSONALITY CHANGED>: "+lisdataset[i].split("_")[len(lisdataset[i].split("_"))-1].upper()+"\n")
        if (inputtext.lower()=="exit"):
          ex=inputtext
          print("<END CHAT>")
          save_chat_log(f,"<END CHAT>"+"\n")
          break
        if(inputtext=="0" or inputtext=="1" or inputtext=="2"):
          change_personality=False
          break
        response= start_chatting(sess, ' '.join(text_preprocess(inputtext)), i,enc_input_train, dec_input_train, targets_train, prediction)  
        print("Chatbot: "+response)
        save_chat_log(f,"Chatbot: "+response+"\n")
write_chat_log()        

<BEGIN CHAT>
<To Change Personality at any time, enter 0 for Comic, 1 for Friend, 2 for Professional, Exit to Exit>
User: Hi
Chatbot: Hello.
User: How are you?
Chatbot: Great, thanks.
User: Do you have any hobbies?
Chatbot: I like you!
User: Are you good at working?
Chatbot: I try, but I don't always get it right.
User: 0
<PERSONALITY CHANGED>: COMIC
User: Don't be oversmart
Chatbot: Eating would require a lot of things I don't have. Like a digestive system. And silverware.
User: You are not very good at conversation
Chatbot: My charms are hard to deny.
User: 1
<PERSONALITY CHANGED>: FRIEND
User: I am depressed
Chatbot: I'm giving you a virtual hug right now.
User: Exit
<END CHAT>
